In [1]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/')
sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/dl')
import dl.loaders.ultrasound_dataset as ultrasound_dataset
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.loaders.mr_us_dataset as mr_us_dataset
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from dl.nets.layers import TimeDistributed

from torchvision import transforms as T

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src/')

from shapeaxi.saxi_layers import AttentionChunk


In [2]:
mount_point = '/mnt/raid/C1_ML_Analysis'

In [ ]:

df = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/C_dataset_analysis_protocoltagsonly_gaboe230_masked_resampled_256_spc075.csv')
df_study = df.query('study_id == "UNC-1037-5"')
print(df_study)

In [4]:
from generative.networks.nets.autoencoderkl import Encoder

latent_dim = 3

    

In [ ]:
vs = us_simu.VolumeSamplingBlindSweep(mount_point=mount_point).cuda()

In [6]:
def sweep_resample(sweep_t, target_shape=(128, 256, 256)):
    return F.interpolate(sweep_t, target_shape, mode='nearest')   

sweeps_fov = []
sweeps_t = []
sweeps_tags = []
sweeps_x_v = []
df_study_tags = df_study[df_study['tag'].isin(vs.tags)]

inverse_transform_us = T.Compose([T.Pad((0, 0, 0, 40)),  T.Lambda(lambda x: T.functional.crop(x, 40, 0, 256, 256))])

for idx, row in df_study_tags.iterrows():
    tag = row['tag']
    
    if tag in vs.tags:
        
        sweep_fn = os.path.join(mount_point, row['file_path'])
        img_np, _ = nrrd.read(sweep_fn, index_order='C')
        img_np = img_np.astype(np.float32)
        sweep_t = torch.tensor(img_np).cuda().float()/255.0
        
        sweep_t = sweep_t.unsqueeze(0).unsqueeze(0)
        
        sweep_t = sweep_resample(sweep_t)
        
        # sweep_t = inverse_transform_us(sweep_t)

        sweeps_tags.append(tag)
        sweeps_t.append(sweep_t)


sweeps_t = torch.cat(sweeps_t)
sweeps_t = sweeps_t[0:8]

# sweeps_fov = torch.stack(sweeps_fov)
# print(sweeps_fov[0].shape)
# print(sweeps_fov[0].shape)
# fig = px.imshow(sweeps_t[0].squeeze().cpu().numpy(), animation_frame=1, binary_string=True)
# fig.show()

# fig = px.imshow(sweeps_fov[0].squeeze().cpu().numpy(), animation_frame=0, binary_string=True)

# fig.show()

In [ ]:

model = us_simu.USPCReconstruction(mount_point=mount_point, lr=1e-4, weight_decay=0.01, latent_channels=3, num_samples=4096, input_dim=6, output_dim=3, K=[27, 27], num_heads=[64, 128], stages=[64, 128], dropout=0.1, pooling_factor=[0.5, 0.5], score_pooling=0, pooling_hidden_dim=[32, 64], feed_forward_hidden_dim=[32, 64], use_skip_connection=0)
model.cuda()

In [8]:
x = model(sweeps_t.cuda(), torch.randint(0, 8, (sweeps_t.shape[0],)).cuda())

In [ ]:
x_ = x.detach().cpu().numpy()[0]
fig = go.Figure(data=[go.Scatter3d(x=x_[:,0], y=x_[:,1], z=x_[:,2], mode='markers', marker=dict(
        size=2,
        color=x_[:,2],                # set color to an array/list of desired values
        # colorscale='jet',   # choose a colorscale
        opacity=1.0
    ))])
fig.show()

In [ ]:
sweeps_t = sweeps_t.permute(0, 1, 4, 3, 2)
x_v = vs.diffusor_tag_resample(sweeps_t, tag=sweeps_tags)

N = 50000

x_v_np0 = x_v.cpu().numpy()[0]
sweeps_t0 = sweeps_t[0].flatten().cpu().numpy()

x_v_np0 = x_v_np0[sweeps_t0 > 0]
sweeps_t0 = sweeps_t0[sweeps_t0 > 0]


random_indices = torch.randperm(x_v_np0.shape[0])[:N]
x_v_np0 = x_v_np0[random_indices]
sweeps_t0 = sweeps_t0[random_indices]

x_v_np1 = x_v.cpu().numpy()[1]
sweeps_t1 = sweeps_t[1].flatten().cpu().numpy()

x_v_np1 = x_v_np1[sweeps_t1 > 0]
sweeps_t1 = sweeps_t1[sweeps_t1 > 0]

random_indices = torch.randperm(x_v_np1.shape[0])[:N]
x_v_np1 = x_v_np1[random_indices]
sweeps_t1 = sweeps_t1[random_indices]


fig = go.Figure(data=[go.Scatter3d(x=x_v_np0[:,0], y=x_v_np0[:,1], z=x_v_np0[:,2], mode='markers', marker=dict(
        size=2,
        color=sweeps_t0,                # set color to an array/list of desired values
        colorscale='jet',   # choose a colorscale
        opacity=1.0
    )), 
    go.Scatter3d(x=x_v_np1[:,0], y=x_v_np1[:,1], z=x_v_np1[:,2], mode='markers', marker=dict(
        size=2,
        color=sweeps_t1,                # set color to an array/list of desired values
        colorscale='jet',   # choose a colorscale
        opacity=1.0
    ))
    ])
fig.show()

In [ ]:
df_train = pd.read_csv(os.path.join(mount_point, 'CSV_files/C1_C2_cines_gt_ga_withmeta_masked_resampled_256_spc075_uuid_study_train_protocoltags.csv'))
df_val = pd.read_csv(os.path.join(mount_point, 'CSV_files/C1_C2_cines_gt_ga_withmeta_masked_resampled_256_spc075_uuid_study_valid_protocoltags.csv'))
df_test = pd.read_csv(os.path.join(mount_point, 'CSV_files/C1_C2_cines_gt_ga_withmeta_masked_resampled_256_spc075_uuid_study_test_protocoltags.csv'))

dm = ultrasound_dataset.USDataModuleBlindSweepWTag(df_train, df_val, df_test, mount_point=mount_point, batch_size=2, num_frames=0, img_column='file_path', ga_column='ga_boe', id_column='study_id', max_sweeps=3)
dm.setup()
# ds = ultrasound_dataset.USDatasetBlindSweepWTag(df_train, mount_point=mount_point, num_frames=0, img_column='file_path', ga_column='ga_boe', transform=None, id_column='study_id', max_sweeps=3)



In [12]:
b0 = next(iter(dm.train_dataloader()))
b0[0].shape